<a href="https://colab.research.google.com/github/NgoMinhToan/colab_massive_data/blob/main/mid_term.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [139]:
pip install pyspark

In [140]:
import re
import time
import random
from itertools import combinations
import requests
from bs4 import BeautifulSoup
import pyspark
import collections

In [141]:
def get_signgles(doc, k, signgles = set()):
    for i in range(0, len(doc) -k):
        signgles.add(doc[i:i+k])
    return signgles

In [142]:
def get_boolean_value(signgles, doc):
    boolean_value = []
    for i in signgles:
        if i in doc:
            boolean_value.append(1)
        else:
            boolean_value.append(0)
    return boolean_value

In [143]:
def Jaccard_similarity(a, b):
    total = 0
    index = 0
    for i in range(len(a)):
        if a[i] == 1 or b[i] == 1:
            total += 1
        if a[i] == b[i] == 1:
            index += 1
    if total == 0:
        return 0
    return index/total

In [144]:
def Probility_Sig(a, b):
    count = 0
    for i in range(len(a)):
        if a[i] == b[i]:
            count += 1
    return count/len(a)

In [145]:
# create hash function
def hash_function(x, a, b, c):
    return (a*x + b) % c

In [146]:
def find_min_prime(x):
    for i in range(length_shingle, length_shingle + 100000):
        prime = True
        for j in range(2, i//2):
            if i%j==0:
                prime = False
                break
        if prime:
            return i

In [147]:
def check_in_bucket(a, z, threshold, min_hash, counts = {}):
    temp = []
    for i1, i2 in combinations(range(a, z), 2):
        prob = Probility_Sig(min_hash[i1], min_hash[i2])
        if prob > threshold:
            temp.append([i1, i2, prob])
            if frozenset([i1, i2]) in list(counts):
                counts[frozenset([i1, i2])] += 1
            else:
                counts[frozenset([i1, i2])] = 1
    return temp

In [148]:
k = 9
num_for_hash = 100
b = 1
time_no_pyspark = 0
time_using_pyspark = 0

#**Crawl dữ liệu**

In [149]:
article1 = []

page = "https://tuoitre.vn/thoi-su.htm"
response = requests.get(page)
soup = BeautifulSoup(response.content, "html.parser")
titles = soup.findAll('h3', class_='title-news')
links = list(set([link.find('a').attrs["href"] for link in titles]))

for link in links:
    full_link = "https://tuoitre.vn" + link
    news = requests.get(full_link)
    soup = BeautifulSoup(news.content, "html.parser")
    title = ''
    image = None
    content = ''
    abstract = ''
    if soup.find("h1", class_="article-title"):
        title = soup.find("h1", class_="article-title").text
    if soup.find("h2", class_="sapo"):
        abstract = soup.find("h2", class_="sapo").text
    body = soup.find("div", id="main-detail-body")
    if body:
        if body.find('p'):
            content = '\n'.join([i.text for i in body.findChildren("p", recursive=False)])
        if body.find("img"):
            image = body.find("img").attrs["src"]
    article1.append({'title': title, 'abstract': abstract, 'body': body, 'content': content, 'image': image, 'link': full_link, 'homePage': 'tuoitre.vn'})


In [150]:
article2 = []

page = "https://vnexpress.net/thoi-su"
response = requests.get(page)
soup = BeautifulSoup(response.content, "html.parser")
titles = soup.findAll('h3', class_='title-news')
links = list(set([link.find('a').attrs["href"] for link in titles]))

for link in links:
    news = requests.get(link)
    soup = BeautifulSoup(news.content, "html.parser")
    title = ''
    content = ''
    image = None
    abstract = ''
    if soup.find("h1", class_="title-detail"):
        title = soup.find("h1", class_="title-detail").text
    if soup.find("p", class_="description"):
        abstract = soup.find("p", class_="description").text
    body = soup.find("article", class_="fck_detail")
    if body:
        if body.find('p'):
            content = '\n'.join([i.text for i in body.findChildren("p", recursive=True)])
        if body.find("img"):
            image = body.find("img").attrs["src"]
    article2.append({'title': title, 'abstract': abstract, 'body': body, 'content': content, 'image': image, 'link': link, 'homePage': 'vnexpress.net'})

In [151]:
article3 = []

page = "https://vietnamnet.vn/vn/thoi-su/"
response = requests.get(page)
soup = BeautifulSoup(response.content, "html.parser")
items = soup.find('div', class_='list-content')
titles = items.findAll('h3')
links = list(set([link.find('a').attrs["href"] for link in titles]))

for link in links:
    full_link = "https://vietnamnet.vn" + link
    news = requests.get(full_link)
    soup = BeautifulSoup(news.content, "html.parser")
    title = ''
    abstract = ''
    content = ''
    image = None
    if soup.find("h1", class_="title"):
        title = soup.find("h1", class_="title").text
    body = soup.find("div", id="ArticleContent")
    if body:
        if body.find("div", class_="ArticleLead"):
            abstract = body.find("div", class_="ArticleLead").text
        if body.find("p"):
            content = '\n'.join([i.text for i in body.findChildren("p", recursive=False)])
        if body.find("img"):
            image = body.find("img").attrs["src"]
    article3.append({'title': title, 'abstract': abstract, 'body': body, 'content': content, 'image': image, 'link': full_link, 'homePage': 'vietnamnet.vn'})

In [152]:
article = list(filter(lambda f: len(f['content']) > 500, article1 + article2 + article3))
documents = [re.sub('[\.,\(\)\n]', '', i['content'].lower()) for i in article]
length_doc = len(documents)
print(len(documents))

73


--------------------------------------------------------
#**Không sử dụng PySpark**

In [153]:
t0 = time.time()

signgles = set()
for doc in documents:
    get_signgles(doc, k, signgles)
length_shingle = len(signgles)

time_no_pyspark += (time.time() - t0)

# print('The Union K-gram:', signgles)
# print()

In [154]:
t0 = time.time()

boolean_vector = []
for doc in documents:
    boolean_vector.append(get_boolean_value(signgles, doc))

time_no_pyspark += (time.time() - t0)

# print('The matrix boolean: ')
# for i in range(len(boolean_vector)):
#     print('C{}:'.format(i+1), boolean_vector[i])
# print()


In [155]:
# Function Hash
hash_func_arr = []

# tim so nguyen to nho nhat nhung lon hon x
c = find_min_prime(length_shingle)

for i in range(num_for_hash):
    hash_func_arr.append([random.randint(1, length_shingle), random.randint(1, length_shingle), c])


In [ ]:
# print('Jaccard similarity: ')
# for i1, i2 in combinations(range(len(boolean_vector)), 2):
#     print('C{} - C{}: '.format(i1+1, i2+1), Jaccard_similarity(boolean_vector[i1], boolean_vector[i2]))
# print()


In [156]:
t0 = time.time()

# Min Hash Using Function
min_hash = [[1000000000 for i in range(num_for_hash)] for j in range(length_doc)]
for r in range(length_shingle):
    hash_row = [hash_function(r, *a) for a in hash_func_arr]
    for c in range(length_doc):
        if boolean_vector[c][r] == 1:
            for i in range(num_for_hash):
                if min_hash[c][i] > hash_row[i]:
                    min_hash[c][i] = hash_row[i]
length_minhash = len(min_hash)

time_no_pyspark += (time.time() - t0)

# print('Min Hash:')
# for i in range(num_for_hash):
#     print(list(map(lambda a: a[i], min_hash)))
# print()


In [157]:
# xuất hiện cùng 1 bucket
t0 = time.time()

counts = {}
print('LSH')
b_r = [[b, int(length_minhash / b)] for b in range(1, length_minhash) if (length_minhash / b) % 1 == 0]
for b, r in b_r:
    threshold = (1/b)**(1/r)
    print('band:',b, ';', 'row:', r, ';', 'threshold: ', threshold)
    index = [[a, z] for a, z in zip(range(0, length_minhash-r+1, r), range(r, length_minhash+1, r))]
    for a, z in index:
        temp = check_in_bucket(a, z, threshold, min_hash, counts)
        for i in temp:
            print('{} - {}: {}'.format(i[0], i[1], i[2]))
            print(min_hash[i[0]])
            print(min_hash[i[1]])
            print(article[i[0]]['link'])
            print(article[i[1]]['link'])

time_no_pyspark += (time.time() - t0)
print('--------------------------------------------------------------------------------------------')
print('Total time execute: ', time_no_pyspark)


LSH
band: 1 ; row: 73 ; threshold:  1.0
--------------------------------------------------------------------------------------------
Total time execute:  58.38241529464722


In [158]:
# số lần xuất hiện trong bucket
for i in list(counts):
    j = list(i)
    print(j[0], j[1])
    print('candidate: ', Probility_Sig(min_hash[j[0]], min_hash[j[1]]))
    print(min_hash[j[0]])
    print(min_hash[j[1]])
    print(j[0], j[1])
    print(article[j[0]]['link'])
    print(article[j[1]]['link'])
    print('exist in band:', counts[i])
    print('--------------------------------------------')

In [159]:
# hiển thị tắt cả các cặp giống nhau với threshold > 0.1
print('All')
threshold = 0.1
for i1, i2 in combinations(range(len(min_hash)), 2):
    if Probility_Sig(min_hash[i1], min_hash[i2]) > threshold:
        print('candidate: ', Probility_Sig(min_hash[i1], min_hash[i2]))
        print(min_hash[i1])
        print(min_hash[i2])
        print(article[i1]['link'])
        print(article[i2]['link'])
        print('--------------------------------------------')

All
candidate:  0.59
[80, 20, 163, 191, 216, 131, 95, 282, 108, 29, 62, 2, 50, 114, 17, 68, 131, 37, 31, 21, 286, 280, 10, 108, 5, 29, 34, 4, 109, 8, 74, 187, 11, 171, 33, 274, 44, 8, 65, 60, 4, 52, 6, 108, 41, 31, 33, 50, 96, 13, 140, 69, 211, 20, 119, 58, 35, 60, 315, 178, 93, 18, 81, 29, 15, 59, 62, 101, 1, 77, 215, 63, 188, 13, 101, 44, 36, 5, 213, 169, 124, 194, 20, 18, 66, 161, 88, 18, 58, 88, 5, 40, 7, 210, 120, 34, 67, 46, 18, 34]
[80, 20, 163, 135, 210, 131, 95, 423, 108, 29, 147, 2, 45, 326, 17, 68, 131, 37, 31, 21, 286, 280, 89, 108, 5, 118, 34, 37, 114, 43, 74, 187, 11, 171, 33, 274, 100, 8, 73, 60, 4, 52, 132, 198, 41, 105, 33, 50, 33, 13, 217, 126, 249, 20, 119, 41, 35, 60, 109, 259, 34, 40, 21, 29, 15, 59, 62, 107, 1, 77, 767, 63, 149, 13, 221, 44, 4, 5, 49, 169, 124, 159, 150, 18, 66, 233, 88, 18, 81, 92, 55, 40, 26, 210, 120, 28, 82, 46, 18, 34]
https://tuoitre.vn/thu-tuong-dong-y-dau-tu-mo-rong-san-bay-dien-bien-co-the-don-may-bay-lon-20210327224434091.htm
https://vie


--------------------------------------------------------
#**Sử dụng PySpark**

In [160]:
conf = pyspark.SparkConf().setMaster('local').setAppName('Similar Document')
sc = pyspark.SparkContext.getOrCreate(conf=conf)

In [161]:
t0 = time.time()

doc = sc.parallelize(documents)
signgles_pyspark = doc.map(lambda a: get_signgles(a, k, set()))
signgles_pyspark = set(signgles_pyspark.reduce(lambda a, b: list(a) + list(b)))
length_shingle = len(signgles_pyspark)

time_using_pyspark += (time.time() - t0)

# print('The Union K-gram:', signgles_pyspark)
# print(length_shingle)

In [162]:
t0 = time.time()

B_vector_pyspark = doc.map(lambda d: get_boolean_value(signgles_pyspark, d))
B_vector = B_vector_pyspark.collect()

time_using_pyspark += (time.time() - t0)

# print('The matrix boolean: ')
# for i in range(len(B_vector)):
#     print('C{}:'.format(i+1), B_vector[i])
# print()

In [163]:
t0 = time.time()

min_hash_pyspark = B_vector_pyspark.map(lambda c: [min([hash_function(i, *hash_func_arr[hash_index]) for i in range(length_shingle) if c[i] == 1] + [1000000000]) for hash_index in range(num_for_hash)])
min_hash = min_hash_pyspark.collect()
length_minhash = len(min_hash)

time_using_pyspark += (time.time() - t0)

# print('Min Hash:')
# for i in range(num_for_hash):
#     print(list(map(lambda a: a[i], min_hash)))
# print()


In [164]:
# xuất hiện cùng 1 bucket
t0 = time.time()

counts = {}
print('LSH')
b_r = [[b, int(length_minhash / b)] for b in range(1, length_minhash) if (length_minhash / b) % 1 == 0]
for b, r in b_r:
    threshold = (1/b)**(1/r)
    print('band:',b, ';', 'row:', r, ';', 'threshold: ', threshold)
    index = [[a, z] for a, z in zip(range(0, length_minhash-r+1, r), range(r, length_minhash+1, r))]
    for a, z in index:
        temp = check_in_bucket(a, z, threshold, min_hash, counts)
        for i in temp:
            print('{} - {}: {}'.format(i[0], i[1], i[2]))
            print(min_hash[i[0]])
            print(min_hash[i[1]])
            print(article[i[0]]['link'])
            print(article[i[1]]['link'])
        
time_using_pyspark += (time.time() - t0)
print('--------------------------------------------------------------------------------------------')
print('Time execute: ', time_using_pyspark)

LSH
band: 1 ; row: 73 ; threshold:  1.0
--------------------------------------------------------------------------------------------
Time execute:  162.5185580253601


In [165]:
# số lần xuất hiện trong bucket
for i in list(counts):
    j = list(i)
    print(j[0], j[1])
    print('candidate: ', Probility_Sig(min_hash[j[0]], min_hash[j[1]]))
    print(min_hash[j[0]])
    print(min_hash[j[1]])
    print(j[0], j[1])
    print(article[j[0]]['link'])
    print(article[j[1]]['link'])
    print('exist in band:', counts[i])
    print('--------------------------------------------')

In [166]:
# hiển thị tắt cả các cặp giống nhau với threshold > 0.1
print('All')
threshold = 0.1
for i1, i2 in combinations(range(len(min_hash)), 2):
    if Probility_Sig(min_hash[i1], min_hash[i2]) > threshold:
        print('candidate: ', Probility_Sig(min_hash[i1], min_hash[i2]))
        print(min_hash[i1])
        print(min_hash[i2])
        print(article[i1]['link'])
        print(article[i2]['link'])
        print('--------------------------------------------')

All
candidate:  0.12
[28, 8, 64, 6, 64, 15, 92, 19, 40, 127, 9, 57, 21, 72, 130, 52, 160, 66, 27, 102, 2, 40, 198, 8, 29, 86, 12, 24, 125, 55, 21, 151, 26, 12, 44, 21, 34, 1, 82, 34, 16, 6, 18, 157, 7, 126, 48, 130, 51, 45, 48, 4, 31, 77, 65, 89, 99, 96, 47, 35, 24, 2, 21, 23, 61, 136, 154, 44, 110, 8, 66, 121, 75, 98, 65, 25, 57, 35, 102, 15, 0, 36, 0, 0, 125, 75, 167, 11, 101, 33, 48, 11, 22, 89, 227, 4, 32, 184, 59, 68]
[124, 23, 266, 360, 95, 123, 150, 111, 40, 48, 4, 98, 294, 331, 25, 164, 4, 132, 29, 197, 311, 5, 327, 260, 3, 446, 12, 83, 50, 62, 481, 7, 149, 97, 213, 72, 314, 9, 11, 136, 16, 378, 18, 13, 114, 99, 48, 65, 48, 24, 9, 51, 31, 75, 104, 378, 30, 108, 383, 204, 118, 35, 21, 153, 106, 73, 154, 112, 143, 254, 65, 479, 3, 60, 51, 25, 27, 12, 101, 184, 99, 30, 41, 204, 174, 79, 397, 37, 18, 53, 200, 28, 27, 89, 227, 371, 32, 44, 104, 240]
https://tuoitre.vn/bat-ngo-vu-2-nguoi-roi-tu-tang-2-dan-khong-biet-chu-dau-tu-coi-noi-roi-cho-thue-2021032817102311.htm
https://tuoitre